In [ ]:
# | default_exp _helpers.utils


In [ ]:
# | export


import os
from typing import *
from urllib.parse import urlparse
from contextlib import contextmanager
from pathlib import Path

import nbdev
from configparser import ConfigParser


In [ ]:
import shutil
from tempfile import TemporaryDirectory

import numpy as np


In [ ]:
# | export


@contextmanager
def set_cwd(cwd_path: Union[Path, str]):

    cwd_path = Path(cwd_path)
    original_cwd = os.getcwd()
    os.chdir(cwd_path)

    try:
        nbdev.config.get_config.cache_clear()
        yield
    finally:
        os.chdir(original_cwd)


In [ ]:
with TemporaryDirectory() as d:
    with set_cwd(d):
        assert (
            Path(os.getcwd()) == Path(d).resolve()
        ), f"{os.getcwd()}, {Path(d).resolve()}"


In [ ]:
# | export


def get_value_from_config(root_path: str, config_name: str) -> str:
    """Get the value from settings.ini file"""

    settings_path = Path(root_path) / "settings.ini"
    config = ConfigParser()
    config.read(settings_path)
    if not config.has_option("DEFAULT", config_name):
        return ""
    return config["DEFAULT"][config_name]


In [ ]:
with TemporaryDirectory() as d:
    settings_path = Path(d) / "settings.ini"
    shutil.copyfile(Path("..") / "settings.ini", settings_path)
    ret_val = get_value_from_config(d, "lib_path")
    print(ret_val)
    assert ret_val == "nbdev_mkdocs", ret_val

    ret_val = get_value_from_config(d, "repo")
    print(ret_val)
    assert ret_val == "nbdev-mkdocs", ret_val

    ret_val = get_value_from_config(d, "user")
    print(ret_val)
    assert ret_val == "airtai", ret_val


nbdev_mkdocs
nbdev-mkdocs
airtai


In [ ]:
# | export


def is_local_path(path):
    # Check if the path is an absolute path
    if os.path.isabs(path):
        return True

    # Check if the path is a URL with a scheme (e.g. http, https, ftp)
    parsed_url = urlparse(path)
    if parsed_url.scheme:
        return False

    # If the path is not an absolute path and does not have a URL scheme,
    # it is assumed to be a local path
    return True


In [ ]:
assert is_local_path("/tmp/abc/file.txt")
assert not is_local_path("http://www.example.com")


In [ ]:
# | export


def add_counter_suffix_to_filename(src_path: Path):
    """Add a counter suffix to the given file

    Args:
        src_path: The path to the file to rename.
    """
    parent_dir = src_path.parent
    counter_suffix = (
        max(
            [
                int(f.stem.split(".")[1])
                for f in parent_dir.glob(f"{src_path.stem}.*.*")
            ],
            default=0,
        )
        + 1
    )
    dst_path = parent_dir / f"{src_path.stem}.{counter_suffix}{src_path.suffix}"
    os.rename(src_path, dst_path)


In [ ]:
with TemporaryDirectory() as d:

    file_path = Path(d) / "social_image.txt"
    with open(file_path, "w") as f:
        f.write("sample_text")

    add_counter_suffix_to_filename(file_path)

    actual = [f.name for f in Path(d).glob(f"{file_path.stem}*.*")]
    print(actual)

    expected = ["social_image.1.txt"]
    np.testing.assert_array_equal(actual, expected)

    assert not file_path.exists()
    assert (Path(d) / "social_image.1.txt").exists()


['social_image.1.txt']


In [ ]:
with TemporaryDirectory() as d:

    for i in [
        "social_image.txt",
        "social_image.1.txt",
        "social_image.2.txt",
        "social_image.11.txt",
    ]:

        file_path = Path(d) / i
        with open(file_path, "w") as f:
            f.write("sample_text")

    file_path = Path(d) / "social_image.txt"
    add_counter_suffix_to_filename(file_path)

    actual = sorted([f.name for f in Path(d).glob(f"{file_path.stem}*.*")])
    print(actual)
    expected = sorted(
        [
            "social_image.1.txt",
            "social_image.2.txt",
            "social_image.11.txt",
            "social_image.12.txt",
        ]
    )
    np.testing.assert_array_equal(actual, expected)

    assert not file_path.exists()
    assert (Path(d) / "social_image.12.txt").exists()


['social_image.1.txt', 'social_image.11.txt', 'social_image.12.txt', 'social_image.2.txt']


In [ ]:
# | export

def unescape_exclamation_mark(s: str) -> str:
    """Replaces the URL-encoded `!%21` character sequence with `!!` in a string.

    Args:
        s: The string to be processed.

    Returns:
        The input string with the `!%21` character sequence replaced with `!!`.
    """
    return s.replace('!%21', '!!')

In [ ]:
_input = """
site_name: Test site
theme:
  name: material
nav:
- Home:
  - Home: index.md
markdown_extensions:
- markdown.extensions.toc:
    slugify: !!python/object/apply:pymdownx.slugs.slugify {kwds: {case: lower}}
    permalink: ''
- markdown.extensions.admonition:
- markdown.extensions.smarty:
    smart_quotes: false
- pymdownx.betterem:
- pymdownx.superfences:
    preserve_tabs: true
    custom_fences:
        # Mermaid diagrams
    - name: diagram
      class: diagram
      format: !%21python/name:pymdownx.superfences.fence_code_format
    - name: math
      class: arithmatex
      format: !%21python/object/apply:pymdownx.arithmatex.arithmatex_fenced_format {
        kwds: {mode: generic, tag: pre}}
    - name: md-render
      class: md-render
      format: !%21python/name:tools.pymdownx_md_render.md_sub_render
- pymdownx.inlinehilite:
    custom_inline:
    - name: math
      class: arithmatex
      format: !!python/object/apply:pymdownx.arithmatex.arithmatex_inline_format {
        kwds: {mode: generic}}
- pymdownx.emoji:
    emoji_index: !%21python/name:materialx.emoji.twemoji
    emoji_generator: !%21python/name:materialx.emoji.to_svg
"""

expected = """
site_name: Test site
theme:
  name: material
nav:
- Home:
  - Home: index.md
markdown_extensions:
- markdown.extensions.toc:
    slugify: !!python/object/apply:pymdownx.slugs.slugify {kwds: {case: lower}}
    permalink: ''
- markdown.extensions.admonition:
- markdown.extensions.smarty:
    smart_quotes: false
- pymdownx.betterem:
- pymdownx.superfences:
    preserve_tabs: true
    custom_fences:
        # Mermaid diagrams
    - name: diagram
      class: diagram
      format: !!python/name:pymdownx.superfences.fence_code_format
    - name: math
      class: arithmatex
      format: !!python/object/apply:pymdownx.arithmatex.arithmatex_fenced_format {
        kwds: {mode: generic, tag: pre}}
    - name: md-render
      class: md-render
      format: !!python/name:tools.pymdownx_md_render.md_sub_render
- pymdownx.inlinehilite:
    custom_inline:
    - name: math
      class: arithmatex
      format: !!python/object/apply:pymdownx.arithmatex.arithmatex_inline_format {
        kwds: {mode: generic}}
- pymdownx.emoji:
    emoji_index: !!python/name:materialx.emoji.twemoji
    emoji_generator: !!python/name:materialx.emoji.to_svg
"""

actual = unescape_exclamation_mark(_input)

print(actual)
assert actual == expected


site_name: Test site
theme:
  name: material
nav:
- Home:
  - Home: index.md
markdown_extensions:
- markdown.extensions.toc:
    slugify: !!python/object/apply:pymdownx.slugs.slugify {kwds: {case: lower}}
    permalink: ''
- markdown.extensions.admonition:
- markdown.extensions.smarty:
    smart_quotes: false
- pymdownx.betterem:
- pymdownx.superfences:
    preserve_tabs: true
    custom_fences:
        # Mermaid diagrams
    - name: diagram
      class: diagram
      format: !!python/name:pymdownx.superfences.fence_code_format
    - name: math
      class: arithmatex
      format: !!python/object/apply:pymdownx.arithmatex.arithmatex_fenced_format {
        kwds: {mode: generic, tag: pre}}
    - name: md-render
      class: md-render
      format: !!python/name:tools.pymdownx_md_render.md_sub_render
- pymdownx.inlinehilite:
    custom_inline:
    - name: math
      class: arithmatex
      format: !!python/object/apply:pymdownx.arithmatex.arithmatex_inline_format {
        kwds: {mode: